In [75]:
import pandas as pd
import yfinance as yf
from tools import sma

Sources:
- https://www.reddit.com/r/algotrading/comments/1cwsco8/a_mean_reversion_strategy_with_211_sharpe/?share_id=CqahUjPGDToYrn9c78uEf&utm_content=1&utm_medium=ios_app&utm_name=ioscss&utm_source=share&utm_term=1

- https://quantitativo.substack.com/p/a-mean-reversion-strategy-with-211

In [76]:
# SYMBOL = "^GSPC"
# SYMBOL = "SPY"

# SYMBOL = "^NDX"
SYMBOL = "QQQ"

stock = yf.download(SYMBOL)

# some data cleaning
stock = stock[~(stock.High == stock.Low) & ~(stock.Open == stock.Close)]
stock = stock.dropna()

[*********************100%%**********************]  1 of 1 completed


In [77]:
def high_low_mean(high: pd.Series, low: pd.Series, intervall: int) -> pd.Series:
    return (high - low).rolling(intervall).mean()

In [78]:
def lower_band(high: pd.Series, hl: pd.Series, factor: float) -> pd.Series:
    return high.rolling(10).max() - factor * hl

In [79]:
# Compute the IBS indicator: (Close - Low) / (High - Low)
stock["ibs"] = (stock.Close - stock.Low) / (stock.High - stock.Low)


stock["sma"] = sma(stock.Close, 300)

In [80]:
from backtesting import Strategy


class mean_reversion(Strategy):
    """
    strategy for trend_trading
    """

    sma_interval = 300
    hl_interval = 25
    factor = 2.5
    ibs_low = 0.3

    def ohlc(self, pos=-1) -> tuple:
        """
        helper function for ohlc data
        """
        return (
            self.data.Open[pos],
            self.data.High[pos],
            self.data.Low[pos],
            self.data.Close[pos],
        )

    def init(self):
        super().init()

        self.ibs = self.I(lambda: self.data.ibs, name="ibs")

        # Compute the rolling mean of High minus Low over the last 25 days
        self.hl_sma = self.I(
            high_low_mean, self.data.High.s, self.data.Low.s, self.hl_interval
        )

        # Compute a lower band as the rolling High over the last 10 days minus 2.5 x the rolling mean of High mins Low (first bullet)
        self.lower_band = self.I(
            lower_band, self.data.High.s, self.hl_sma.s, self.factor
        )

        # regime filter
        self.sma = self.I(sma, self.data.Close.s, self.sma_interval)

    def next(self):
        # super().next()

        _open, _high, _low, _close = self.ohlc()

        # trade management for existing trade
        for trade in self.trades:
            if trade.is_long:
                # update trailing stop
                trade.sl = max(trade.sl, self.sma[-1])

                # close on next day open
                if _close > self.data.High[-2]:
                    trade.close()

        if len(self.trades) == 0:
            if (
                (_close < self.lower_band[-1])
                and (self.ibs[-1] < self.ibs_low)
                and (_low > self.sma[-1])
            ):
                self.buy(sl=self.sma[-1])

In [81]:
from backtesting import Backtest

bt = Backtest(
    stock["2000-01-01":],
    mean_reversion,
    cash=100_000,
    commission=0.002,
    trade_on_close=True,
)
stats = bt.run()
bt.plot(superimpose=False)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-21 00:00:00
Duration                   8905 days 00:00:00
Exposure Time [%]                    13.90304
Equity Final [$]                232049.065423
Equity Peak [$]                 235783.465498
Return [%]                         132.049065
Buy & Hold Return [%]              381.055396
Return (Ann.) [%]                     3.54756
Volatility (Ann.) [%]                7.701678
Sharpe Ratio                         0.460622
Sortino Ratio                         0.69315
Calmar Ratio                          0.21134
Max. Drawdown [%]                  -16.786003
Avg. Drawdown [%]                    -2.48917
Max. Drawdown Duration     1134 days 00:00:00
Avg. Drawdown Duration      108 days 00:00:00
# Trades                                  185
Win Rate [%]                        69.189189
Best Trade [%]                        3.62981
Worst Trade [%]                     -7.897493
Avg. Trade [%]                    

In [82]:
stats, heatmap = bt.optimize(
    sma_interval=range(50, 400, 25),
    hl_interval=range(5, 50, 5),
    maximize="Equity Final [$]",  # "Profit Factor"
    max_tries=500,
    random_state=0,
    return_heatmap=True,
)

/Users/produktmanagement/Python/github/backtesting-trading-strategies/.venv/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


In [83]:
# heatmap
heatmap.sort_values().iloc[-10:]

sma_interval  hl_interval
250           45             243785.039899
175           45             244000.995377
250           10             245568.536210
200           45             245980.364250
225           10             246108.046910
175           10             246801.869067
250           40             247500.675844
175           40             247757.881091
200           10             249303.996484
              40             249757.293097
Name: Equity Final [$], dtype: float64

In [84]:
# Recommendation
stats["_strategy"]

<Strategy mean_reversion(sma_interval=200,hl_interval=40)>

In [85]:
stats = bt.run(**stats._strategy._params)
bt.plot(superimpose=False, open_browser=True)
stats

Start                     2000-01-03 00:00:00
End                       2024-05-21 00:00:00
Duration                   8905 days 00:00:00
Exposure Time [%]                   13.147083
Equity Final [$]                249757.293097
Equity Peak [$]                 265020.024061
Return [%]                         149.757293
Buy & Hold Return [%]              381.055396
Return (Ann.) [%]                    3.863402
Volatility (Ann.) [%]                7.149616
Sharpe Ratio                         0.540365
Sortino Ratio                        0.798855
Calmar Ratio                         0.185758
Max. Drawdown [%]                  -20.798048
Avg. Drawdown [%]                   -2.066313
Max. Drawdown Duration     1551 days 00:00:00
Avg. Drawdown Duration       83 days 00:00:00
# Trades                                  183
Win Rate [%]                        71.584699
Best Trade [%]                        3.62981
Worst Trade [%]                    -14.046924
Avg. Trade [%]                    